<h2>Load data</h2>

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
import time

from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

# Disable the extensive logging info
tf.logging.set_verbosity(tf.logging.ERROR)

# Import data
mnist = input_data.read_data_sets('data_mnist', one_hot=True)

Extracting data_mnist/train-images-idx3-ubyte.gz
Extracting data_mnist/train-labels-idx1-ubyte.gz
Extracting data_mnist/t10k-images-idx3-ubyte.gz
Extracting data_mnist/t10k-labels-idx1-ubyte.gz


<h2>Linear classifier</h2>

In [5]:
# start time
start = time.time()

# Create the model
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.matmul(x, W) + b

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])

# Softmax loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
# Train using Gradient Descent
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# Start interactive session
sess = tf.InteractiveSession()
# Init variables
tf.global_variables_initializer().run()

# Train
for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

# Test trained model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images,y_: mnist.test.labels})

# Stop interactive session
sess.close()

# stop times
end = time.time()

# print results
print("Test Accuracy: {0:0.2f}%".format(test_accuracy * 100))
print("Time elapsed:  {0:0.2f} sec".format(end - start))

Test Accuracy: 91.64%
Time elapsed:  1.39 sec


<h2>Deep Convolutional Network</h2>

Build the ConvNet

In [2]:
def deepnn(x):
  """deepnn builds the graph for a deep net for classifying digits.
  Args:
    x: an input tensor with the dimensions (N_examples, 784), where 784 is the
    number of pixels in a standard MNIST image.
  Returns:
    A tuple (y, keep_prob). y is a tensor of shape (N_examples, 10), with values
    equal to the logits of classifying the digit into one of 10 classes (the
    digits 0-9). keep_prob is a scalar placeholder for the probability of
    dropout.
  """
  # Reshape to use within a convolutional neural net.
  # Last dimension is for "features" - there is only one here, since images are
  # grayscale -- it would be 3 for an RGB image, 4 for RGBA, etc.
  with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

  # First convolutional layer - maps one grayscale image to 32 feature maps.
  with tf.name_scope('conv1'):
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

  # Pooling layer - downsamples by 2X.
  with tf.name_scope('pool1'):
    h_pool1 = max_pool_2x2(h_conv1)

  # Second convolutional layer -- maps 32 feature maps to 64.
  with tf.name_scope('conv2'):
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

  # Second pooling layer.
  with tf.name_scope('pool2'):
    h_pool2 = max_pool_2x2(h_conv2)

  # Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
  # is down to 7x7x64 feature maps -- maps this to 1024 features.
  with tf.name_scope('fc1'):
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

  # Dropout - controls the complexity of the model, prevents co-adaptation of
  # features.
  with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

  # Map the 1024 features to 10 classes, one for each digit
  with tf.name_scope('fc2'):
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])

    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
  return y_conv, keep_prob


def conv2d(x, W):
  """conv2d returns a 2d convolution layer with full stride."""
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
  """max_pool_2x2 downsamples a feature map by 2X."""
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


def weight_variable(shape):
  """weight_variable generates a weight variable of a given shape."""
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)


def bias_variable(shape):
  """bias_variable generates a bias variable of a given shape."""
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

Init and run the network

In [3]:
# start timer
start = time.time()

# Create the model
x = tf.placeholder(tf.float32, [None, 784])

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])

# Build the graph for the deep net
y_conv, keep_prob = deepnn(x)

# Softmax loss function
with tf.name_scope('loss'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=y_conv)
    cross_entropy = tf.reduce_mean(cross_entropy)

# Train using Adam Optimizer
with tf.name_scope('adam_optimizer'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Evaluate accuracy
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
accuracy = tf.reduce_mean(correct_prediction)

# Start interactive session
sess = tf.InteractiveSession()
# Init variables
tf.global_variables_initializer().run()

# Train
for i in range(5001):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
        print('Step %d - training accuracy %.1f%%' % (i, train_accuracy*100))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

# Test trained model
test_accuracy = accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})

# Stop interactive session
sess.close()

# stop times
end = time.time()

# print results
print("\nTest Accuracy: {0:0.2f}%".format(test_accuracy * 100))
print("Time elapsed:  {0:0.2f} sec".format(end - start))

Step 0 - training accuracy 14.0%
Step 100 - training accuracy 88.0%
Step 200 - training accuracy 84.0%
Step 300 - training accuracy 100.0%
Step 400 - training accuracy 86.0%
Step 500 - training accuracy 98.0%
Step 600 - training accuracy 96.0%
Step 700 - training accuracy 92.0%
Step 800 - training accuracy 94.0%
Step 900 - training accuracy 92.0%
Step 1000 - training accuracy 92.0%
Step 1100 - training accuracy 92.0%
Step 1200 - training accuracy 100.0%
Step 1300 - training accuracy 96.0%
Step 1400 - training accuracy 100.0%
Step 1500 - training accuracy 100.0%
Step 1600 - training accuracy 96.0%
Step 1700 - training accuracy 96.0%
Step 1800 - training accuracy 100.0%
Step 1900 - training accuracy 92.0%
Step 2000 - training accuracy 100.0%
Step 2100 - training accuracy 96.0%
Step 2200 - training accuracy 96.0%
Step 2300 - training accuracy 100.0%
Step 2400 - training accuracy 98.0%
Step 2500 - training accuracy 98.0%
Step 2600 - training accuracy 98.0%
Step 2700 - training accuracy 96.